# 認識歷史回測

當我們在市場上交易時，若是對當前市場的趨勢變動沒有把握時，就需要歷史回測(Back Testing)模擬真實的開盤狀況，借此來驗證自己的交易策略是否是有效的。回測不僅僅只是數學計量模型的計算，也包含了計量模型以外的市場分析。

以往大家都沒有完整的歷史數據、只有零散在各處的新聞媒體或券商的統計、盤後資料時，是很難進行歷史回測的。而現在由於資料取得容易，計量回測就成了一個不可或缺的工具。

# 先 import 我們所需要的模組

In [ ]:
import xlwings as xw

In [ ]:
# 打開你的 tsmc_back_test.xlsx 檔案
wb = xw.Book(r"放置 tsmc_back_test.xlsx 的路徑")

In [ ]:
tsmc_sheet = wb.sheets['2330']
tsmc_sheet

# 計算加權移動平均

## 動態截取資料的筆數

In [ ]:
# 會 VBA 的人，馬上可以看出 xlwings 原汁原味的實作了 Python 版的 End() 函數
from xlwings.constants import Direction

# 從 B1 儲存格開始，往下查找到最後一個有值的儲存格
last_cell = tsmc_sheet.range('A1').end(Direction.xlDown)
# 把從 A1 開始，最右下角的儲存格底色換成紅色
last_cell.color = (255, 0, 0)
# 截取該儲存格的 row 值
last_row = last_cell.row
last_row
# 這樣我們就不會因爲試算表資料的改變而需要頻繁的修改程式碼

## 動態設定 Excel 儲存格或範圍的名稱

```python
# 可以用 name 屬性定義一個儲存格的名稱
cells(1, 'A').name = 'hello'
# 同樣的，range 也有 name 屬性
range('A1:A3').name = 'world'
```

In [ ]:
# 設定我們的範例試算表上的名稱
tsmc_sheet.range('K2:K11').name = 'weight10d'
tsmc_sheet.range('K2:K6').name = 'weight5d'

## 接下來我們算出短期 (5日) 與長期 (10日) 的移動平均

In [ ]:
# 5日加權移動平均計算
for i in range(6, last_row+1):
    # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
    formula = "=SUM(B{}:B{}*weight5d)/SUM(weight5d)".format(i-4, i)
    # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
    tsmc_sheet.cells(i, 'C').formula_array = formula

In [ ]:
# 10日加權移動平均計算
for i in range(11, last_row+1):
    # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
    formula = "=SUM(B{}:B{}*weight10d)/SUM(weight10d)".format(i-9, i)
    # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
    tsmc_sheet.cells(i, 'D').formula_array = formula

## 流程控制

![flow control](https://www.dropbox.com/s/01oyzdlgiyafmzo/flow_control.jpg?raw=1)
當我們遇上程式需要針對不同選項執行不同結果時，我們需要學會用程式語言表達 Yes 和 No，以及邏輯上的分支點

## Boolean (布林值)
布林值和整數一樣為資料型別，只有兩個值：

```python
True # 代表成立
False # 代表不成立
```

---
## 比較運算子

| 運算子 | 功能     |
| :----: | :------: |
| ==     | 等於     |
| !=     | 不等於   |
| <      | 小於     |
| >      | 大於     |
| <=     | 小於等於 |
| >=     | 大於等於 |

---
## 範例

```python
'hello' == 'hello'
# True
47 > 47
# False
47 >= 47
# True
```

---
## if 陳述句
若 `if` 後面的比較式條件成立，也就是結果為 `True`，則執行子句中的程式碼：

```python
myName = "Bob"

if myName == "Bob":
    print("my name is Bob!")
```

---
## if...else 陳述句

若 `if` 關鍵字後面的比較式結果為 `False`，則忽略 `if` 子句，執行 `else` 的子句：

```python
myName = "Alice"

if myName == "Bob":
    print("my name is Bob!")
else:
    print("my name is {}!".format(myName))
```

---
## elif 陳述句
白話文就是指 "否則如果"

當可能性/選項超出兩種時，可用：

```python
name = "Mary"

if name == "Bob":
    print("Hello Bob!")
elif name == "Alice":
    print("Hello Alice!")
elif name == "Mary":
    print("Hello Mary!")
else
    print("Hello Stranger!")
```

只要 `if` 後面的條件結果是 `False`，便會看 `elif` 後面的條件，
若比較式結果為 `True`，便會執行 `elif` 的子句，不然便會繼續
檢查下一個 `elif`，若全部的比較式結果都是 `False`，則執行 `else`
底下的程式碼

---
## and, or, not

![](https://www.dropbox.com/s/nf5bu3bx9tx6luv/logical.png?dl=1)

# 我們的交易策略

我們用加權移動平均分別算出一個長期與短期的量化指標，再利用這兩個指標判斷進出場的時機：

- 如果 :五日均價 > 十日均價 = **買入一張**
- 如果 :五日均價 < 十日均價 = **賣出一張**

***備注：非投資建議，本課程提供的資料及交易策略，只可作為參考用途，學員在投資前，務請運用個人獨立思考做出抉擇，如因此招致任何損失，概與本課程無涉。**


## 我們先透過此策略，手動計算第一筆資料：

| 日期        | 收盤價    | 5日加權 | 10日加權 |買入股數 |賣出股數|持有股數|持有資金|總資產|
| ----------- |:--------:|:-------:|:-------:|:------:|:-----:|:------:|:-----:|-----:|
| 2017/10/20  | 237.5    | 237.6667| 236.590 |1000    |0      |1000    |762500 |1000000|

In [ ]:
# 計算第一天的交易 2017/10/20

tsmc_sheet.cells(11, 'E').value = 1000
tsmc_sheet.cells(11, 'F').value = 0
tsmc_sheet.cells(11, 'G').value = 1000
tsmc_sheet.cells(11, 'H').value =  tsmc_sheet.cells(18, 'L').value - tsmc_sheet.cells(11, 'B').value * 1000
tsmc_sheet.cells(11, 'I').value = tsmc_sheet.cells(11, 'H').value +  tsmc_sheet.cells(11, 'B').value * tsmc_sheet.cells(11, 'G').value

In [ ]:
# 實作交易策略

for i in range(12, last_row+1):
    # 截取當天的 5日加權移動平均
    short_term_ma = tsmc_sheet.cells(i, 'C').value
    # 截取當天的 10日加權移動平均
    long_term_ma = tsmc_sheet.cells(i, 'D').value
    # 截取當天收盤價
    price_today = tsmc_sheet.cells(i, 'B').value
    # 若 5日 > 10日，而且我有足夠買入以今日收盤價計價的 1000 股的現金，就買入 1000 股（在 E 欄顯示 1000）
    if (short_term_ma > long_term_ma) and (tsmc_sheet.cells(i-1, 'H').value > price_today * 1000):
        tsmc_sheet.cells(i, 'E').value = 1000
    else:
    # 若上述條件不符和，就買入 0 股，（在 E 欄顯示 0）
        tsmc_sheet.cells(i, 'E').value = 0
    # 若 10日 > 5日，而且昨天的持有股數大於 1000 股，就賣出 1000 股

    if (long_term_ma > short_term_ma) and (tsmc_sheet.cells(i-1, 'G').value >= 1000):
        tsmc_sheet.cells(i, 'F').value = 1000
    else:
        tsmc_sheet.cells(i, 'F').value = 0
    # 持有股數，算法是前一天的持有股數 + 今天的買入股數 - 今天的賣出股數
    tsmc_sheet.cells(i, 'G').value = tsmc_sheet.cells(i-1, 'G').value + tsmc_sheet.cells(i, 'E').value - tsmc_sheet.cells(i, 'F').value
    # 持有資金，算法是前一天的持有資金 + 今日收盤價 x (今天的賣出股數 - 今天的買入股數)
    tsmc_sheet.cells(i, 'H').value = tsmc_sheet.cells(i-1, 'H').value + price_today * (tsmc_sheet.cells(i, 'F').value - tsmc_sheet.cells(i, 'E').value)
    # 總資產則是持有股數 x 今日收盤價 + 今日持有資金
    tsmc_sheet.cells(i, 'I').value = tsmc_sheet.cells(i, 'H').value + tsmc_sheet.cells(i, 'G').value * price_today

# 計算并且將總收益顯示在 L20
tsmc_sheet.cells(20, 'L').value = tsmc_sheet.cells(last_row, 'I').value - tsmc_sheet.cells(18, 'L').value